# Langgraph notebook 1
https://langchain-ai.github.io/langgraph/

Building agents from graphs

In [1]:
# The basics

from dotenv import load_dotenv
import os
import json
import re
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Adding tools

### Retriever tool

In [2]:
# Initialize vector database of Opentrons documentation

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../rag-documentation/opentrons-docs.pdf")
documents = loader.load()

def clean_text(text):
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

cleaned_docs = [Document(page_content=clean_text(doc.page_content), metadata=doc.metadata) for doc in documents]
splits = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200
        ).split_documents(cleaned_docs)
vector = FAISS.from_documents(splits, OpenAIEmbeddings())
retriever = vector.as_retriever(search_kwargs={"k": 3})

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_opentrons_docs",
    "Search for documentation about the Python Opentrons v2 API",
)

In [3]:
tools = [retriever_tool]

## Langgraph stuff

In [6]:
from langchain_core.tools import tool
from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI

tool_node = ToolNode(tools)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

In [7]:
from typing import Annotated, Literal, TypedDict

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

In [8]:
# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [9]:
# Define a new graph
workflow = StateGraph(MessagesState)

In [10]:
# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [11]:
# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

In [12]:
# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

In [13]:
# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

In [14]:
# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

In [15]:
# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

In [16]:
from langchain_core.messages import HumanMessage

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="How do I blow out small drops of liquid from the pipette tip on an Opentrons robot?")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'To blow out small drops of liquid from the pipette tip on an Opentrons robot, you can use the `InstrumentContext.blow_out()` method. Here’s how you can do it:\n\n1. **Basic Blow Out**: If you want to blow out air from the pipette\'s current position, simply call:\n   ```python\n   pipette.blow_out()\n   ```\n\n2. **Blow Out to a Specific Well**: If you want to blow out to a specific well (for example, a trash container), you can specify the well:\n   ```python\n   pipette.blow_out(plate["B1"])  # Replace "B1" with your desired well\n   ```\n\n3. **Blow Out to Trash Container**: To blow out into the pipette\'s trash container, use:\n   ```python\n   pipette.blow_out(pipette.trash_container)\n   ```\n\n4. **Touch Tip**: Additionally, you can use the `InstrumentContext.touch_tip()` method to move the pipette so that the tip touches the walls of a well, which helps knock off any droplets that might cling to the tip:\n   ```python\n   pipette.touch_tip()\n   ```\n\nThese methods can be use

In [17]:
print(final_state["messages"][-1].content)

To blow out small drops of liquid from the pipette tip on an Opentrons robot, you can use the `InstrumentContext.blow_out()` method. Here’s how you can do it:

1. **Basic Blow Out**: If you want to blow out air from the pipette's current position, simply call:
   ```python
   pipette.blow_out()
   ```

2. **Blow Out to a Specific Well**: If you want to blow out to a specific well (for example, a trash container), you can specify the well:
   ```python
   pipette.blow_out(plate["B1"])  # Replace "B1" with your desired well
   ```

3. **Blow Out to Trash Container**: To blow out into the pipette's trash container, use:
   ```python
   pipette.blow_out(pipette.trash_container)
   ```

4. **Touch Tip**: Additionally, you can use the `InstrumentContext.touch_tip()` method to move the pipette so that the tip touches the walls of a well, which helps knock off any droplets that might cling to the tip:
   ```python
   pipette.touch_tip()
   ```

These methods can be used in your protocol to ens